<a name="data"></a>
## 1. Data Preparation

Original data at https://rdrr.io/cran/openintro/man/loans_full_schema.html

In [1]:
!wget https://raw.githubusercontent.com/rickysoo/lendingclub/main/loans_full_schema.csv

--2021-06-28 08:25:04--  https://raw.githubusercontent.com/rickysoo/lendingclub/main/loans_full_schema.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2548030 (2.4M) [text/plain]
Saving to: ‘loans_full_schema.csv.1’

loans_full_schema.c 100%[===================>]   2.43M  --.-KB/s    in 0.1s    

2021-06-28 08:25:05 (17.4 MB/s) - ‘loans_full_schema.csv.1’ saved [2548030/2548030]



In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

np.set_printoptions(suppress = True)

In [3]:
df = pd.read_csv('loans_full_schema.csv', index_col = 0)
df.head()

,emp_length,state,homeownership,annual_income,verified_income,debt_to_income,annual_income_joint,verification_income_joint,debt_to_income_joint,delinq_2y,months_since_last_delinq,earliest_credit_line,inquiries_last_12m,total_credit_lines,open_credit_lines,total_credit_limit,total_credit_utilized,num_collections_last_12m,num_historical_failed_to_pay,months_since_90d_late,current_accounts_delinq,total_collection_amount_ever,current_installment_accounts,accounts_opened_24m,months_since_last_credit_inquiry,num_satisfactory_accounts,num_accounts_120d_past_due,num_accounts_30d_past_due,num_active_debit_accounts,total_debit_limit,num_total_cc_accounts,num_open_cc_accounts,num_cc_carrying_balance,num_mort_accounts,account_never_delinq_percent,tax_liens,public_record_bankrupt,loan_purpose,application_type,loan_amount,term,interest_rate,installment,grade,sub_grade,issue_month,loan_status,initial_listing_status,disbursement_method,balance,paid_total,paid_principal,paid_interest,paid_late_fees
emp_title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
global config engineer,3.0,NJ,MORTGAGE,90000.0,Verified,18.01,NaN,NaN,NaN,0,38.0,2001,6,28,10,70795,38767,0,0,38.0,0,1250,2,5,5.0,10,0.0,0,2,11100,14,8,6,1,92.9,0,0,moving,individual,28000,60,14.07,652.53,C,C3,Mar-2018,Current,whole,Cash,27015.86,1999.33,984.14,1015.19,0.0
warehouse office clerk,10.0,HI,RENT,40000.0,Not Verified,5.04,NaN,NaN,NaN,0,NaN,1996,1,30,14,28800,4321,0,1,NaN,0,0,0,11,8.0,14,0.0,0,3,16500,24,14,4,0,100.0,0,1,debt_consolidation,individual,5000,36,12.61,167.54,C,C1,Feb-2018,Current,whole,Cash,4651.37,499.12,348.63,150.49,0.0
assembly,3.0,WI,RENT,40000.0,Source Verified,21.15,NaN,NaN,NaN,0,28.0,2006,4,31,10,24193,16000,0,0,28.0,0,432,1,13,7.0,10,0.0,0,3,4300,14,8,6,0,93.5,0,0,other,individual,2000,36,17.09,71.40,D,D1,Feb-2018,Current,fractional,Cash,1824.63,281.80,175.37,106.43,0.0
customer service,1.0,PA,RENT,30000.0,Not Verified,10.16,NaN,NaN,NaN,0,NaN,2007,0,4,4,25400,4997,0,1,NaN,0,0,1,1,15.0,4,0.0,0,2,19400,3,3,2,0,100.0,1,0,debt_consolidation,individual,21600,36,6.72,664.19,A,A3,Jan-2018,Current,whole,Cash,18853.26,3312.89,2746.74,566.15,0.0
security supervisor,10.0,CA,RENT,35000.0,Verified,57.96,57000.0,Verified,37.66,0,NaN,2008,7,22,16,69839,52722,0,0,NaN,0,0,1,6,4.0,16,0.0,0,10,32700,20,15,13,0,100.0,0,0,credit_card,joint,23000,36,14.07,786.87,C,C3,Mar-2018,Current,whole,Cash,21430.15,2324.65,1569.85,754.80,0.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, global config engineer  to operations analyst
Data columns (total 54 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   emp_length                        9183 non-null   float64
 1   state                             10000 non-null  object 
 2   homeownership                     10000 non-null  object 
 3   annual_income                     10000 non-null  float64
 4   verified_income                   10000 non-null  object 
 5   debt_to_income                    9976 non-null   float64
 6   annual_income_joint               1495 non-null   float64
 7   verification_income_joint         1455 non-null   object 
 8   debt_to_income_joint              1495 non-null   float64
 9   delinq_2y                         10000 non-null  int64  
 10  months_since_last_delinq          4342 non-null   float64
 11  earliest_credit_line              100

In [5]:
df.isnull().sum()

emp_length                           817
state                                  0
homeownership                          0
annual_income                          0
verified_income                        0
debt_to_income                        24
annual_income_joint                 8505
verification_income_joint           8545
debt_to_income_joint                8505
delinq_2y                              0
months_since_last_delinq            5658
earliest_credit_line                   0
inquiries_last_12m                     0
total_credit_lines                     0
open_credit_lines                      0
total_credit_limit                     0
total_credit_utilized                  0
num_collections_last_12m               0
num_historical_failed_to_pay           0
months_since_90d_late               7715
current_accounts_delinq                0
total_collection_amount_ever           0
current_installment_accounts           0
accounts_opened_24m                    0
months_since_las

Select only numerical columns.

In [6]:
df = df.select_dtypes(include = ['int64', 'float64'])

Remove unnecessary columns with missing values.

In [7]:
df.drop(['annual_income_joint', 'debt_to_income_joint', 'months_since_last_delinq', 'months_since_90d_late', 'months_since_last_credit_inquiry', 'num_accounts_120d_past_due'], axis = 1, inplace = True)

Fill in missing values with averages.

In [8]:
df['emp_length'].fillna(value = df['emp_length'].mean(), inplace = True)
df['debt_to_income'].fillna(value = df['debt_to_income'].mean(), inplace = True)

In [9]:
features = df.columns.to_list()
features.remove('loan_amount')
target = ['loan_amount']

df = df[features + target]
df.reset_index(inplace = True, drop = True)
df.head()

,emp_length,annual_income,debt_to_income,delinq_2y,earliest_credit_line,inquiries_last_12m,total_credit_lines,open_credit_lines,total_credit_limit,total_credit_utilized,num_collections_last_12m,num_historical_failed_to_pay,current_accounts_delinq,total_collection_amount_ever,current_installment_accounts,accounts_opened_24m,num_satisfactory_accounts,num_accounts_30d_past_due,num_active_debit_accounts,total_debit_limit,num_total_cc_accounts,num_open_cc_accounts,num_cc_carrying_balance,num_mort_accounts,account_never_delinq_percent,tax_liens,public_record_bankrupt,term,interest_rate,installment,balance,paid_total,paid_principal,paid_interest,paid_late_fees,loan_amount
0,3.0,90000.0,18.01,0,2001,6,28,10,70795,38767,0,0,0,1250,2,5,10,0,2,11100,14,8,6,1,92.9,0,0,60,14.07,652.53,27015.86,1999.33,984.14,1015.19,0.0,28000
1,10.0,40000.0,5.04,0,1996,1,30,14,28800,4321,0,1,0,0,0,11,14,0,3,16500,24,14,4,0,100.0,0,1,36,12.61,167.54,4651.37,499.12,348.63,150.49,0.0,5000
2,3.0,40000.0,21.15,0,2006,4,31,10,24193,16000,0,0,0,432,1,13,10,0,3,4300,14,8,6,0,93.5,0,0,36,17.09,71.40,1824.63,281.80,175.37,106.43,0.0,2000
3,1.0,30000.0,10.16,0,2007,0,4,4,25400,4997,0,1,0,0,1,1,4,0,2,19400,3,3,2,0,100.0,1,0,36,6.72,664.19,18853.26,3312.89,2746.74,566.15,0.0,21600
4,10.0,35000.0,57.96,0,2008,7,22,16,69839,52722,0,0,0,0,1,6,16,0,10,32700,20,15,13,0,100.0,0,0,36,14.07,786.87,21430.15,2324.65,1569.85,754.80,0.0,23000


In [10]:
df.isnull().sum()

emp_length                      0
annual_income                   0
debt_to_income                  0
delinq_2y                       0
earliest_credit_line            0
inquiries_last_12m              0
total_credit_lines              0
open_credit_lines               0
total_credit_limit              0
total_credit_utilized           0
num_collections_last_12m        0
num_historical_failed_to_pay    0
current_accounts_delinq         0
total_collection_amount_ever    0
current_installment_accounts    0
accounts_opened_24m             0
num_satisfactory_accounts       0
num_accounts_30d_past_due       0
num_active_debit_accounts       0
total_debit_limit               0
num_total_cc_accounts           0
num_open_cc_accounts            0
num_cc_carrying_balance         0
num_mort_accounts               0
account_never_delinq_percent    0
tax_liens                       0
public_record_bankrupt          0
term                            0
interest_rate                   0
installment   

In [11]:
df.shape

(10000, 36)

In [12]:
df.to_csv('loans_full_schema_small.csv', index = False)

<a name="weka"></a>
## 2. Objective Function - Weka

We use Weka to obtain the objective function.

In [13]:
# import os
# import sys

# sys.path
# sys.path.append("/usr/lib/jvm/java-11-openjdk-amd64/bin/")
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64/"

# !apt-get install build-essential python3-dev
# !apt-get install python3-pil python3-pygraphviz
# !apt install openjdk-11-jdk

!pip3 install javabridge --no-cache-dir
!pip3 install python-weka-wrapper3 --no-cache-dir

In [14]:
import weka.core.jvm as jvm
jvm.start()

DEBUG:weka.core.jvm:Adding bundled jars
DEBUG:weka.core.jvm:Classpath=['/usr/local/lib/python3.7/dist-packages/javabridge/jars/rhino-1.7R4.jar', '/usr/local/lib/python3.7/dist-packages/javabridge/jars/runnablequeue.jar', '/usr/local/lib/python3.7/dist-packages/javabridge/jars/cpython.jar', '/usr/local/lib/python3.7/dist-packages/weka/lib/python-weka-wrapper.jar', '/usr/local/lib/python3.7/dist-packages/weka/lib/weka.jar']
DEBUG:weka.core.jvm:MaxHeapSize=default
DEBUG:weka.core.jvm:Package support disabled


In [15]:
from weka.classifiers import Classifier
import weka.core.converters as converters

Load data into Weka.

In [16]:
data = converters.load_any_file('loans_full_schema_small.csv')
data

@relation loans_full_schema_small

@attribute emp_length numeric
@attribute annual_income numeric
@attribute debt_to_income numeric
@attribute delinq_2y numeric
@attribute earliest_credit_line numeric
@attribute inquiries_last_12m numeric
@attribute total_credit_lines numeric
@attribute open_credit_lines numeric
@attribute total_credit_limit numeric
@attribute total_credit_utilized numeric
@attribute num_collections_last_12m numeric
@attribute num_historical_failed_to_pay numeric
@attribute current_accounts_delinq numeric
@attribute total_collection_amount_ever numeric
@attribute current_installment_accounts numeric
@attribute accounts_opened_24m numeric
@attribute num_satisfactory_accounts numeric
@attribute num_accounts_30d_past_due numeric
@attribute num_active_debit_accounts numeric
@attribute total_debit_limit numeric
@attribute num_total_cc_accounts numeric
@attribute num_open_cc_accounts numeric
@attribute num_cc_carrying_balance numeric
@attribute num_mort_accounts numeric
@att

Build a regression model using Linear Regression.

In [17]:
from weka.classifiers import Classifier, Evaluation
from weka.core.classes import Random
data.class_is_last()   # set class attribute
classifier = Classifier(classname="weka.classifiers.functions.LinearRegression")
classifier.build_classifier(data)

Show the resulting objective function.

In [18]:
print(classifier)


Linear Regression Model

loan_amount =

     -7.0273 * open_credit_lines +
      0.0001 * total_credit_limit +
      5.671  * num_satisfactory_accounts +
      0.0003 * total_debit_limit +
     15.3506 * term +
    -14.4662 * interest_rate +
      1.729  * installment +
      0.9459 * balance +
      0.9452 * paid_principal +
   -418.7843


Evaluate the model.

In [19]:
evaluation = Evaluation(data)
evaluation.crossvalidate_model(classifier, data, 10, Random(1))
print(evaluation.summary())


Correlation coefficient                  0.9994
Mean absolute error                     66.5097
Root mean squared error                351.9618
Relative absolute error                  0.782  %
Root relative squared error              3.4164 %
Total Number of Instances            10000     



<a name="python"></a>
## 3. Objective Function - Python

We confirm the objective function by running it using Scikit-learn in Python.

In [20]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

Take relevant variables generated in Weka as features.

In [21]:
features = ['open_credit_lines', 'total_credit_limit', 'num_satisfactory_accounts', 'total_debit_limit', 'term', 'interest_rate', 'installment', 'balance', 'paid_principal']

In [22]:
X = df[features]
y = df[target]

In [23]:
X_ = PolynomialFeatures(degree = 1, include_bias = False).fit_transform(X)
X_.shape

(10000, 9)

In [24]:
lr = LinearRegression().fit(X_, y)

In [25]:
intercept, coefficients = lr.intercept_, lr.coef_
intercept, coefficients

(array([-418.78428186]),
 array([[ -7.02734747,   0.00007239,   5.67100956,   0.00029692,
          15.35058187, -14.46617336,   1.7289671 ,   0.94588167,
           0.94521545]]))

The results match the ones produced in Weka.